## DIA 113: Crear un Historial de Reportes por Usuario con Enlaces de Descarga


Hoy aprenderás a construir una vista en Flask donde cada usuario puede ver su historial de reportes generados, con fechas, tipo de archivo (PDF o CSV), y enlaces para descargar directamente desde Firebase (o tu servidor).

✅ ¿Qué aprenderás hoy?

🧠 Crear un modelo de historial de reportes

📁 Guardar metadatos de cada archivo generado

🌐 Mostrar el historial en un dashboard

🔗 Permitir descarga directa del archivo

🧩 1. Crear el modelo Reporte
Agrega este nuevo modelo a tu models.py (o dentro de app.py si no estás usando uno separado):

python
Copiar
Editar
class Reporte(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    fecha_generado = db.Column(db.DateTime, default=datetime.utcnow)
    tipo = db.Column(db.String(10))  # 'pdf' o 'csv'
    url = db.Column(db.String(500))
⚠️ No olvides ejecutar db.create_all() o una migración si estás usando Flask-Migrate.

🧩 2. Guardar registro cada vez que se genere un reporte
Modifica tu función que sube archivos a Firebase o genera localmente:

python
Copiar
Editar
def registrar_reporte(user, url, tipo):
    nuevo_reporte = Reporte(
        user_id=user.id,
        tipo=tipo,
        url=url
    )
    db.session.add(nuevo_reporte)
    db.session.commit()
Llama a esta función después de generar y subir el archivo:

python
Copiar
Editar
registrar_reporte(user, url_descarga, tipo)
🧩 3. Crear la ruta Flask para ver historial
python
Copiar
Editar
@app.route('/historial-reportes')
@login_required
def historial_reportes():
    reportes = Reporte.query.filter_by(user_id=current_user.id).order_by(Reporte.fecha_generado.desc()).all()
    return render_template('historial_reportes.html', reportes=reportes)
🧩 4. Template historial_reportes.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8">
  <title>📁 Historial de Reportes</title>
  <style>
    table { border-collapse: collapse; width: 80%; margin: 20px auto; }
    th, td { border: 1px solid #ccc; padding: 10px; text-align: center; }
    th { background-color: #f2f2f2; }
  </style>
</head>
<body>
  <h2 style="text-align:center;">📄 Historial de Reportes Generados</h2>
  <table>
    <thead>
      <tr>
        <th>Fecha</th>
        <th>Tipo</th>
        <th>Enlace</th>
      </tr>
    </thead>
    <tbody>
      {% for reporte in reportes %}
      <tr>
        <td>{{ reporte.fecha_generado.strftime('%Y-%m-%d %H:%M') }}</td>
        <td>{{ reporte.tipo.upper() }}</td>
        <td><a href="{{ reporte.url }}" target="_blank">📥 Descargar</a></td>
      </tr>
      {% else %}
      <tr><td colspan="3">No hay reportes generados aún.</td></tr>
      {% endfor %}
    </tbody>
  </table>
</body>
</html>
